<a href="https://colab.research.google.com/github/lustraka/puzzles/blob/main/AoC2021/AoC_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advent of Code Puzzles
[Advent of Code 2021](https://adventofcode.com/2021) | [reddit/adventofcode](https://www.reddit.com/r/adventofcode/)

In [1]:
import requests
import pandas as pd
import numpy as np
import re
path = 'https://raw.githubusercontent.com/lustraka/puzzles/main/AoC2021/data/'

## [Day 22](https://adventofcode.com/2021/day/22): Reactor Reboot
### Part I
- **Unknown**: The number of cubes set on.
- **Data**: A sequence of reboot steps.
- **Condition**:
  - Consider x, y, z positions of at least -50 and at most 50.
- **Plan**:
  - Parse the input

In [2]:
example = """on x=10..12,y=10..12,z=10..12
on x=11..13,y=11..13,z=11..13
off x=9..11,y=9..11,z=9..11
on x=10..10,y=10..10,z=10..10"""

In [3]:
def parse(input):
  """Load steps into lists."""
  steps = [row.split() for row in input.split('\n')]
  out = []
  for step in steps:
    limits = [x[0].split('..') for x in re.findall(r'[xyz]=(.*?)(,|$)', step[1])]
    limit_int = []
    for limit in limits:
      limit_int.append([int(limit[0]), int(limit[1])])
    out.append([step[0],np.array(limit_int)])

  return out

steps = parse(example)
steps

[['on', array([[10, 12],
         [10, 12],
         [10, 12]])], ['on', array([[11, 13],
         [11, 13],
         [11, 13]])], ['off', array([[ 9, 11],
         [ 9, 11],
         [ 9, 11]])], ['on', array([[10, 10],
         [10, 10],
         [10, 10]])]]

In [ ]:
[x[0].split('..') for x in re.findall(r'[xyz]=(.*?)(,|$)', 'x=10..11,y=-10..-12,z=100..120')]

[['10', '11'], ['-10', '-12'], ['100', '120']]

In [ ]:
def get_range(limits, coor):
  if coor == 'x':
    return range(max(limits[0][0],-50)+50, min(limits[0][1],50)+51)
  elif coor == 'y':
    return range(max(limits[1][0],-50)+50, min(limits[1][1],50)+51)
  else:
    return range(max(limits[2][0],-50)+50, min(limits[2][1],50)+51)

In [ ]:
# Use max(axis=0) to get max() of [coor1, coor2]
a = np.array([[1,2],[3,4],[5,6]])
for i in [None, 0, 1]:
  print(i, a.max(axis=i), a.min(axis=i))

None 6 1
0 [5 6] [1 2]
1 [2 4 6] [1 3 5]


In [ ]:
# Use broadcasting to shift coordinates
a+[50,51]

array([[51, 53],
       [53, 55],
       [55, 57]])

In [4]:
def trim_shift_limits(limits):
  max = limits.max(axis=0)
  min = limits.min(axis=0)
  # Skip limits out of range in any dimension
  if max[0] > 51 or min[1] < -51:
    return np.array([])
  for coor in limits:
    if coor[0] < -50:
      coor[0] = -50
    if coor[1] > 50:
      coor[1] = 50
  return limits+[50,51]

In [ ]:
example0 = """on x=-100..0,y=-100..120,z=10..12
on x=-110..-100,y=11..13,z=11..13
off x=45..100,y=9..110,z=9..11
on x=100..1000,y=10..10,z=10..10"""
steps = parse(example0)
for step in steps:
  print(trim_shift_limits(step[1]))

[[  0  51]
 [  0 101]
 [ 60  63]]
[]
[[ 95 101]
 [ 59 101]
 [ 59  62]]
[]


In [ ]:
reactor = np.zeros((101,101,101)).astype(int)
reactor.shape

(101, 101, 101)

In [ ]:
for step in steps:
  limits = trim_shift_limits(step[1])
  #print(1 if step[0] == 'on' else 0, limits)
  if limits.any():
    print(limits.shape)
    for x in range(*limits[0]):
      for y in range(*limits[1]):
        for z in range(*limits[2]):
          reactor[x,y,z] = 1 if step[0] == 'on' else 0
reactor[reactor>0].size

(3, 2)
(3, 2)
(3, 2)
(3, 2)


39

In [ ]:
np.array([]).any()

False

In [5]:
def solve_part1(input):
  steps = parse(input)
  reactor = np.zeros((101,101,101)).astype(int)
  for step in steps:
    limits = trim_shift_limits(step[1])
    if limits.any():
      for x in range(*limits[0]):
        for y in range(*limits[1]):
          for z in range(*limits[2]):
            reactor[x,y,z] = 1 if step[0] == 'on' else 0
  return reactor[reactor>0].size
solve_part1(example)

39

In [6]:
example2 = """on x=-20..26,y=-36..17,z=-47..7
on x=-20..33,y=-21..23,z=-26..28
on x=-22..28,y=-29..23,z=-38..16
on x=-46..7,y=-6..46,z=-50..-1
on x=-49..1,y=-3..46,z=-24..28
on x=2..47,y=-22..22,z=-23..27
on x=-27..23,y=-28..26,z=-21..29
on x=-39..5,y=-6..47,z=-3..44
on x=-30..21,y=-8..43,z=-13..34
on x=-22..26,y=-27..20,z=-29..19
off x=-48..-32,y=26..41,z=-47..-37
on x=-12..35,y=6..50,z=-50..-2
off x=-48..-32,y=-32..-16,z=-15..-5
on x=-18..26,y=-33..15,z=-7..46
off x=-40..-22,y=-38..-28,z=23..41
on x=-16..35,y=-41..10,z=-47..6
off x=-32..-23,y=11..30,z=-14..3
on x=-49..-5,y=-3..45,z=-29..18
off x=18..30,y=-20..-8,z=-3..13
on x=-41..9,y=-7..43,z=-33..15
on x=-54112..-39298,y=-85059..-49293,z=-27449..7877
on x=967..23432,y=45373..81175,z=27513..53682"""

solve_part1(example2)

590784

In [7]:
r = requests.get(path+'AoC2021_22.txt')
solve_part1(r.text[:-1])

576028

`736684` : That's not the right answer; your answer is too high. 
> Check determination of ranges in the cuboids (`step[1]`)

In [ ]:
steps = parse(r.text[:-1])
steps[-3:]

[['on', array([[-16269,   4565],
         [-54781, -37040],
         [ 54733,  74868]])], ['on', array([[ 31918,  53358],
         [ 52954,  77962],
         [-20309,  -9155]])], ['off', array([[ 6259, 30795],
         [63250, 73684],
         [25841, 49676]])]]

### Part II
- **Unknown**: 
- **Data**: 
- **Condition**:
  - 
  - 
- **Plan**:
  - 
